In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

When doing EDA, I would usually like to performe the following steps:

* Carefully read the description from person/entity that provided the data to understand the features and the task at hand
* Visualize the data, quite alot of it! This is to get a sense of distribution of each feature and the lables/target values. We can also detect and fill out or disregard missing data in this step
* Look at correlations between features and that between features and labels/target values. This gives me a sense of which features are more important, which ones can be (potentially) ignored all together.
You might think we should never delete features, the more data the merrier, but that is not always the case, especially when using models with low to moderatelearning capacity. 
* Perform outlier detection and removal, testing the possibility of dimensionality reduction
* Convert  categorical features to numerical features (in most cases this is necessary), standardization of data and saving to be used fo training

Of course the first step is to always load the data and take a first look at the number of features that are provided. Because the data is very large, in the interest of saving time, we sample the training data and the testing data and only use 10% of these for EDA. When sampling data for EDA we need to ensure the sampled data is representative of the entire dataset.

In [ ]:
# for the EDA I only use 20% of the training data, randomly sampled from original dataset, I have saved this in a different .csv file to make
# re-running of this notebook faster, for the first run un-comment the following line
#train_data = pd.read_csv('../input/train.csv').sample(frac=0.2, axis=0)
train_data = pd.read_csv('../input/springleaf-train-small/train_small.csv')
train_data.drop(['Unnamed: 0'], axis=1, inplace=True)  # this feature was created during subsampling
original_feat_dim = train_data.shape[1]
train_data.info()

In [ ]:
train_data.head(10)

In [ ]:
# looking at categorical columns
categorical = train_data.select_dtypes(include=[np.object]).columns
categorical, len(categorical)

In [ ]:
# looking at numerical columns
numerical = train_data.select_dtypes(include=[np.number]).columns
numerical, len(numerical)

Looks like we have 1883 numerical features (1882 really counting out ID). We are aiming to predict the 'target'. Due to the large number of features involved, this data is a prime candidate for dimensionality reduction. Lets put a pin on this and do some visualization! First stop: histogram of targets, this helps us make sure we don't have a class imbalance problem.

In [ ]:
plt.hist(train_data["target"])

In [ ]:
print("Ratio of positive samples (target == 1) to negative samples (target == 0) is {}"\
          .format(len(train_data[train_data["target"] == 1]) / len(train_data[train_data["target"] == 0])))

**Note:** This means any classifier that always predicts 0 will get an accuracy of about 70%. This is a good performance benchmark to begin with. 

# Dealling with missing data and data cleaning

Before looking at the features and trying to make sense out of them, we will first look at missing data, constant features and duplicate features.

First, lets look at missing data and see how big of a problem that is.

In [ ]:
null_count = train_data.isnull().sum(axis=0).sort_values(ascending=False)
null_count.head(30)

In [ ]:
# figuring out how many features have more than 10% of data missing them
np.sum(null_count > 0.1 * train_data.shape[0])

In [ ]:
# I think its safe to remove these features, I will create a to_remove list to store all features that need to be dropped
missing = [feature for feature in train_data.columns if null_count[feature] > 0.1 * train_data.shape[0]]
train_data.drop(missing, axis=1, inplace=True)

**Dealing with the rest of missing data**

We removed the features where more than 10% of data is missing. Lets look at whats left

In [ ]:
# taking a second look
total = train_data.isnull().sum().sort_values(ascending=False)
percent = total / train_data.shape[0]
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Of the ones left, we adapt the following simple rule of thumb (we can always reconsider this)

- If lest than 1% of data is missing of a given feature, just remove the training sample that is missing this feature (all of the features above except VAR_0212)

- Any feature missing more thant 1% of data should be "completed". If the feature is numerical, we replace the NAN with mean of the feature across the training data set. Otherwise, if feature is categorical, we replace it with the mode. (more complicated extrapolation methods are abundant but we will keep things simple for now) (feature VAR_0212 above)

In [ ]:
remove_missing_examples = [feat for feat in percent.index if percent[feat] < 0.01]
fill_missing_examples = [feat for feat in percent.index if percent[feat] > 0.01]

# filling in the missing data for features containing many of them
for col in fill_missing_examples:
    if col in categorical:
        # fill missing data with mode
        train_data[col].fillna(train_data[col].mode(), inplace=True)
    else:
        # fill missing data with mean
        train_data[col].fillna(train_data[col].mean(), inplace=True)

# removing rows with missing data in them (only a few examples will be deleted at this point)
train_data.dropna(axis=0, inplace=True)

In [ ]:
# final check for missing data
train_data.isnull().sum().max()

**Deleting constant features**

Another set of features that need to be removed are the ones with constant values. We need to get rid of those too!

In [ ]:
values_count = train_data.nunique(dropna=False).sort_values()
np.sum(values_count == 1)

In [ ]:
# noting features with constant values for removal
constants = [feature for feature in train_data.columns if values_count[feature] == 1]
train_data.drop(constants, axis=1, inplace=True)

****Deleting duplicate features****

Finally, we need to get rid of features that are duplicates of another one, this may happen very rarely but when there are a large number of features I always look for this situation. To this end, instead of comparing all features with all other features, I fist compose a list of possible duplicates by comparing mean and std values. If they are the same then I will perform element-wise comparison. 

But first, lets encode categorical features into numerical features so that we can find meaningfull mean and std values. This encoding also helps us ignore the missing data when comparing duplicates, for instances, a missing piece of information might be represented differently in different columns. 

In [ ]:
enc_data =  pd.DataFrame(index=train_data.index)
for col in train_data.columns:
    enc_data[col] = train_data[col].factorize()[0]

In [ ]:
# noting and dropping duplicated features 
duplicates = []
data_mean = np.mean(enc_data)
data_std = np.std(enc_data)
for i, ref in enumerate(train_data.columns[:-1]):
    for other in train_data.columns[i + 1:-1]:
        if other not in duplicates and data_mean[ref] == data_mean[other] and \
           data_std[ref] == data_std[other] and np.all(enc_data[ref] == enc_data[other]):
            duplicates.append(other)    
train_data.drop(duplicates, axis=1, inplace=True)
len(duplicates)

In [ ]:
print("Dimension of feature space before data cleaning : %d" % original_feat_dim)
print("Dimension of feature space after data cleaning : %d" % train_data.shape[1])
print("Number of features removed : %d" % (original_feat_dim - train_data.shape[1]))

# Going through data

Alright time to look at data in hops of getting a better understanding. What we learn here will be subsequently used in another data preparation script. Here is the plan:

1 - look at categorical data to see if we can create new features, especially feature crosses from them

2 - Find correlation between numerical data and targets, identify top correlated features and look at their distributions, this will tell us if outlier detection or binning of features in needed

3 - Do the same for categorical fetures, to find correlation we have to first encode these 

In [ ]:
categorical = train_data.select_dtypes(include=[np.object]).columns
numerical = train_data.select_dtypes(include=[np.number]).columns

In [ ]:
train_data[categorical].describe()

Observations: 
    - VAR_0226, VAR_0230 and VAR_0236 are also nearly dominated by their most frequent values so these can be removed too (making note but not removing these yet), this will reduce the dimension of the feature space,

In [ ]:
potential_to_remove = ['VAR_0226', 'VAR_0230', 'VAR_0236']

In [ ]:
train_data[categorical].head(20)

Observations:
    - Features : VAR_0075, VAR_0204, VAR_0217 correspond to dates, keep this in mind when converting these features into numreical ones
    - It looks like VAR_0217 is larger than VAR_0075, we may be able use the difference between the two as a good feature, lets check this.
    - VAR_0204 dates mostly fall into late January and early February of 2014, given the date of the competition, this looks like a "last login by user" kinda feature, might be irrelevant to target, we have to check how it changes vs. target.
    - Features VAR_200, VAR_0237, VAR_274 corrspond to location, first one is name of a city, the other two correspond to state
    - Features VAR_0404, VAR_466, VAR_0467 and VAR_0493 are largely dominated by -1, have to check their histograms.

In [ ]:
dates = ['VAR_0075', 'VAR_0204', 'VAR_0217']
locations = ['VAR_200', 'VAR_0237', 'VAR_274']

date1 = pd.to_datetime(train_data["VAR_0075"],format = '%d%b%y:%H:%M:%S')
date2 = pd.to_datetime(train_data["VAR_0217"],format = '%d%b%y:%H:%M:%S')
np.all(date2 > date1)

In [ ]:
train_data[['VAR_0404', 'VAR_0466', 'VAR_0467', 'VAR_0493']].nunique(dropna=False)

In [ ]:
train_data['VAR_0466'].value_counts() / train_data.shape[0]

In [ ]:
train_data['VAR_0467'].value_counts() / train_data.shape[0]

- I think -1 here indicates missing data in original data acquisition process, these features do not seem to carry much information.
- Also, 'Discharge NA' may be considered as an outlier value for feature 'VAR_0467'.
Lets keep a note of this and look at the correlation between categorical data and the target. 

In [ ]:
# converting categorical features to encoded variables first, 
# for features represnting date, we separate year, month day, hour and minute, this is a very crude represntation, we may 
# need more interesting time features like time of the day, weekday and so on
from datetime import datetime
year_func = lambda x: datetime.strptime(x, "%d%b%y:%H:%M:%S" ).year
month_func = lambda x: datetime.strptime(x, "%d%b%y:%H:%M:%S" ).month
day_func = lambda x: datetime.strptime(x, "%d%b%y:%H:%M:%S" ).day
hour_func = lambda x: datetime.strptime(x, "%d%b%y:%H:%M:%S" ).hour
minute_func = lambda x: datetime.strptime(x, "%d%b%y:%H:%M:%S" ).minute

enc_data =  pd.DataFrame(index=train_data.index)
for col in categorical:
    enc_data[col] = train_data[col].factorize()[0]
    if col in dates:
        enc_data[col + '_year'] = train_data[col].map(year_func)
        enc_data[col + '_month'] = train_data[col].map(month_func)
        enc_data[col + '_day'] = train_data[col].map(day_func)
        enc_data[col + '_hour'] = train_data[col].map(hour_func)
        enc_data[col + '_minute'] = train_data[col].map(minute_func)
expanded_categoricals = list(enc_data.columns) # saving which variables are categorical for possible one hot encoding 
enc_data["target"] = train_data["target"]
# finding correlation and looking at it
corrmat = enc_data.corr()
corrmat["target"].sort_values(ascending=False)

It looks like different components of the date have different significance in predicting the target.
Also, some of the features that we created are constant and need to be removed. Lets remove these, find correlation matrix again and plot it as a heatmap

In [ ]:
enc_data.drop(['VAR_0075_hour', 'VAR_0075_minute', 'VAR_0204_year', 'VAR_0217_hour', 'VAR_0217_minute'], axis=1, inplace=True)
corrmat = enc_data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=1, square=True)

Observations:
    - There is no significant correlation between any categorical feature and the target, however, this deos not mean we can disregard these features.
      Note: correlation only shows linear dependance between two random quantities
    - VAR_0466 and VAR_0467 seem to be highly correlated, in fact, if we go back and look at the first 20 rows we notice that VAR_0466 has 'l' in each row where VAR_0467 has 'Discharged', lets test this out, if this is the case. Dimensionality reduction will take care of this!

Now lets look at the correlation between numerical features and the target, because we have a large number of numerical features, this might take a while!

In [ ]:
numerical = train_data.select_dtypes(include=[np.number]).columns
corrmat2 = pd.concat([train_data[numerical], train_data['target']], axis = 0).corr()
corrmat2["target"].sort_values(ascending=False)

Alright it looks like, unlike with categorical variables, the numerical variables show some decent correlation levels with the target. Lets 
    - look at distributions of some these feautres, looking for outliers
    - check correlations between features themselves and see how many features have a correlation of larger than 0,8 (in absolute value terms)

In [ ]:
# getting more important features
treshold = 0.17
importants = [feat for feat in corrmat2.columns if abs(corrmat2['target'].loc[feat]) > treshold]

# randomly selecting 6 features from importants
sample_feats = np.random.choice(importants[:-1], 6)

# looking at histograms
f, ax = plt.subplots(figsize=(20, 10))
for i in range(len(sample_feats)):
    plt.subplot(2, 3, i + 1)
    sns.distplot(train_data[sample_feats[i]])

Apparently some outlier detection is needed! These histograms have a very long tail. We may also have to quantize these features into bins. For now lets look at how target changes vs these sample features.

In [ ]:
f, ax = plt.subplots(figsize=(20, 10))
f.suptitle('Features With High Correlation (6 randomly chosen ones)', size=14)
for i in range(len(sample_feats)):
    plt.subplot(2, 3, i + 1)
    sns.boxplot(x='target', y=sample_feats[i], data=train_data)

Finally, lets count the highly correlated numerical features.

In [ ]:
high_corrs = []
for i, c1 in enumerate(corrmat2.columns):
    for c2 in corrmat2.columns[i + 1:]:
        if abs(corrmat2[c1].loc[c2]) > 0.8:
            high_corrs.append((c1, c2))
len(high_corrs)

We can see that the features are highly correlated, some dimensionality reduction is surely needed! 
Before doing so, we save all the info that we can use for data preparation in a separate script.


In [ ]:
np.savez('./prep_info.npz', missing, fill_missing_examples, remove_missing_examples, constants, duplicates,
         dates, locations, potential_to_remove, expanded_categoricals)

# Dimensionality reduction and visualization
In this section I will perform dimensionality reduction to embed the data in a 2D space, this gives us an idea of how separable the data is, and, if, embedding into a lower dimenison helps.
I try the following three methods:
    - Principal component analysis (and kernel PCA)
    - tSNE
    - Diffusion mapping (with two different choices for affinity matrix)

In [ ]:
from sklearn.manifold import TSNE
from sklearn.manifold import SpectralEmbedding
from sklearn.decomposition import PCA, KernelPCA, FastICA

In [ ]:
# first step, separating features and targets, remember that we have already encoded categorical data and stored the result in enc_data 
num_samples = 1000 # number of samples to use from each class of target for dimensionality reduction, reduce this if you are impatient!
numerical_feats = train_data[numerical].drop(['target'], axis=1, inplace=False)
new_data = pd.concat([numerical_feats, enc_data], axis=1)
positive_samples = new_data[new_data['target'] == 1].sample(num_samples)
negative_samples = new_data[new_data['target'] == 0].sample(num_samples)
to_transform = pd.concat([positive_samples, negative_samples]).sample(frac=1).reset_index(drop=True)
to_transform.head(20)

In [ ]:
features = to_transform.drop(['target'], axis=1, inplace=False)
labels = to_transform['target']

**First method: the humble PCA**

In [ ]:
pca_embedding =  PCA(n_components=2) 
pca_emb_data = pca_embedding.fit_transform(features.values)
plt.figure(figsize=(15,15))
plt.scatter(pca_emb_data[labels == 1, 0], pca_emb_data[labels == 1, 1], color='red', label='positive samples')
plt.scatter(pca_emb_data[labels == 0, 0], pca_emb_data[labels == 0, 1], color='blue', label='negative samples')
plt.legend()

**First method (b) : kernel PCA with rbf kernel**

In [ ]:
kpca_embedding =  KernelPCA(n_components=2, kernel='rbf')
kpca_emb_data = kpca_embedding.fit_transform(features.values)
plt.figure(figsize=(15,15))
plt.title('Reduced data with kernel PCA (RBF kernel)')
plt.scatter(kpca_emb_data[labels == 1, 0], kpca_emb_data[labels == 1, 1], color='red', label='positive samples')
plt.scatter(kpca_emb_data[labels == 0, 0], kpca_emb_data[labels == 0, 1], color='blue', label='negative samples')
plt.legend()

**Second method: tSNE**

In [ ]:
tsne_embedding =  TSNE(n_components=2) 
tsne_emb_data = tsne_embedding.fit_transform(features.values)
plt.figure(figsize=(15,15))
plt.title('Reduced data with tSNE')
plt.scatter(tsne_emb_data[labels == 1, 0], tsne_emb_data[labels == 1, 1], color='red', label='positive samples')
plt.scatter(tsne_emb_data[labels == 0, 0], tsne_emb_data[labels == 0, 1], color='blue', label='negative samples')
plt.legend()

**Third method (a): diffusion map with RBF kernel**

In [ ]:
spec_embedding = SpectralEmbedding(n_components=2, affinity='rbf')
transformed_data2 = spec_embedding.fit_transform(features.values)
fig = plt.figure(figsize=(30,10))
plt.subplot(1, 3, 1)
plt.scatter(transformed_data2[labels == 1, 0], transformed_data2[labels == 1, 1], color='red', label='positive samples')
plt.legend()
plt.subplot(1, 3, 2)
plt.scatter(transformed_data2[labels == 0, 0], transformed_data2[labels == 0, 1], color='blue', label='negative samples')
plt.legend()
plt.subplot(1, 3, 3)
plt.scatter(transformed_data2[labels == 1, 0], transformed_data2[labels == 1, 1], color='red', label='positive samples')
plt.scatter(transformed_data2[labels == 0, 0], transformed_data2[labels == 0, 1], color='blue', label='negative samples')
plt.legend()

**Third method (b): diffusion map with nearest neighbours used for affinity**

In [ ]:
spec_embedding2 = SpectralEmbedding(n_components=2, affinity='nearest_neighbors', n_neighbors=30)
transformed_data2 = spec_embedding2.fit_transform(features.values)
fig = plt.figure(figsize=(30,10))
plt.subplot(1, 3, 1)
plt.scatter(transformed_data2[labels == 1, 0], transformed_data2[labels == 1, 1], color='red', label='positive samples')
plt.legend()
plt.subplot(1, 3, 2)
plt.scatter(transformed_data2[labels == 0, 0], transformed_data2[labels == 0, 1], color='blue', label='negative samples')
plt.legend()
plt.subplot(1, 3, 3)
plt.scatter(transformed_data2[labels == 1, 0], transformed_data2[labels == 1, 1], color='red', label='positive samples')
plt.scatter(transformed_data2[labels == 0, 0], transformed_data2[labels == 0, 1], color='blue', label='negative samples')
plt.legend()

**Conclusions and next steps**

* The dimension of the feature space is very large but features are highly correlated so dimensionality reduction might help
* There are alot of duplicate feautres, constant features and features with alot of missing entires, these need to be removed
* There are a few fetures related to the date, we can use the differences between these dates as a new feature! 
* The categoircal features do not show promising correlation with the target value, but the numerical features do
* Outlier data detection and removal may need to be carried out (not in the first version of the model we build though)
* Data visualisation in 2D space shows that this is a chellnging data set that may not be easily separable